# How to match? 

## 0. Import libraries 

In [1]:
import pandas as pd
import warnings
import numpy as np
from lightautoml.addons.hypex import Matcher

warnings.filterwarnings('ignore')
%config Completer.use_jedi = False

'nlp' extra dependecy package 'gensim' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'nltk' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'transformers' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'gensim' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'nltk' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'transformers' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.


/Users/20224761/PycharmProjects/matcher/lightautoml/ml_algo/dl_model.py:41: UserWarning: 'transformers' - package isn't installed
  warnings.warn("'transformers' - package isn't installed")
/Users/20224761/PycharmProjects/matcher/lightautoml/text/nn_model.py:22: UserWarning: 'transformers' - package isn't installed
  warnings.warn("'transformers' - package isn't installed")
/Users/20224761/PycharmProjects/matcher/lightautoml/text/dl_transformers.py:25: UserWarning: 'transformers' - package isn't installed
  warnings.warn("'transformers' - package isn't installed")


## 1. Create or upload your dataset  
In this case we will create random dataset with known effect size  
If you have your own dataset, go to the part 2 


In [2]:
# Simulating dataset with known effect size
num_users = 10000
num_months = 12

signup_months = np.random.choice(np.arange(1, num_months), num_users) * np.random.randint(0,2, size=num_users) # signup_months == 0 means customer did not sign up
df = pd.DataFrame({
    'user_id': np.repeat(np.arange(num_users), num_months),
    'signup_month': np.repeat(signup_months, num_months), # signup month == 0 means customer did not sign up
    'month': np.tile(np.arange(1, num_months+1), num_users), # months are from 1 to 12
    'spend': np.random.poisson(500, num_users*num_months) #np.random.beta(a=2, b=5, size=num_users * num_months)*1000 # centered at 500
})
# A customer is in the treatment group if and only if they signed up
df["treat"] = df["signup_month"]>0
# Simulating an effect of month (monotonically decreasing--customers buy less later in the year)
df["spend"] = df["spend"] - df["month"]*10
# Simulating a simple treatment effect of 100
after_signup = (df["signup_month"] < df["month"]) & (df["treat"])
df.loc[after_signup,"spend"] = df[after_signup]["spend"] + 100
df.head()

,user_id,signup_month,month,spend,treat
0,0,5,1,478,True
1,0,5,2,457,True
2,0,5,3,463,True
3,0,5,4,454,True
4,0,5,5,448,True


In [3]:
# Setting the signup month (for ease of analysis)
i = 3
df_i_signupmonth = (
    df[df.signup_month.isin([0, i])]
    .groupby(["user_id", "signup_month", "treat"])
    .apply(
        lambda x: pd.Series(
            {
                "pre_spends": x.loc[x.month < i, "spend"].mean(),
                "post_spends": x.loc[x.month > i, "spend"].mean(),
            }
        )
    )
    .reset_index()
)
df_i_signupmonth

,user_id,signup_month,treat,pre_spends,post_spends
0,1,0,False,491.0,416.000000
1,3,0,False,500.5,425.555556
2,5,0,False,488.5,419.000000
3,6,0,False,471.0,406.888889
4,7,0,False,502.5,423.666667
...,...,...,...,...,...
5470,9990,0,False,491.0,408.555556
5471,9991,0,False,474.0,408.111111
5472,9995,3,True,485.0,510.222222
5473,9998,0,False,470.5,426.111111


In [4]:
# Additional category features
gender = np.random.choice(a=[0,1], size=df_i_signupmonth.user_id.nunique())
age = np.random.choice(a=range(18, 70), size=df_i_signupmonth.user_id.nunique())
industry = np.random.choice(a=range(1, 3), size=df_i_signupmonth.user_id.nunique())
df_i_signupmonth['age'] = age
df_i_signupmonth['is_male'] =  gender
df_i_signupmonth['industry'] =  industry
df_i_signupmonth['industry'] = df_i_signupmonth['industry'].astype('str')
df_i_signupmonth['treat'] = df_i_signupmonth['treat'].astype(int)
df_i_signupmonth.head()

,user_id,signup_month,treat,pre_spends,post_spends,age,is_male,industry
0,1,0,0,491.0,416.000000,33,1,1
1,3,0,0,500.5,425.555556,40,0,2
2,5,0,0,488.5,419.000000,60,0,2
3,6,0,0,471.0,406.888889,69,1,2
4,7,0,0,502.5,423.666667,34,1,1


In [5]:
df_i_signupmonth.columns

Index(['user_id', 'signup_month', 'treat', 'pre_spends', 'post_spends', 'age',
       'is_male', 'industry'],
      dtype='object')

## 2. Matching  
### 2.0 Init params
info_col used to define informative attributes that should not be part of matching, such as user_id  
But to explicitly store this column in the table, so that you can compare directly after computation

In [6]:
info_col = ['user_id', 'signup_month']

outcome = 'post_spends'
treatment = 'treat'

### 2.1 Simple matching
This is the easiest way to initialize and calculate metrics on a Matching task  
Use it when you are clear about each attribute or if you don't have any additional task conditions (Strict equality for certain features) 

In [7]:
# Standard model with base parameters
model = Matcher(input_data=df_i_signupmonth, outcome=outcome, treatment=treatment, info_col=info_col)
results, quality_results, df_matched = model.estimate()

  0%|          | 0/5475 [00:00<?, ?it/s]

### 2.2 Matching with a fixed variable  
Used when you have categorical feature(s) that you want to compare by strict equality  
group_col is used for strict comparison of categorical features.  
In our case there is only one attribute  
If there are several such attributes, you should make one of them and use it

In [8]:
# group_col is used for strict comparison of categorical features.  
# In our case there is only one attribute  
# If there are several such attributes, you should make one of them and use it

group_col = "industry"

In [9]:
model = Matcher(input_data=df_i_signupmonth, outcome=outcome, treatment=treatment,
                info_col=info_col, group_col=group_col)
results, quality_results, df_matched = model.estimate()

  0%|          | 0/4 [00:00<?, ?it/s]

### 2.3 Matching but you don't know which features are better to use  
This is the easiest way to initialize and calculate metrics on a Matching task  
Use this method when you want to select the most important features and make matching based on them. 

In [10]:
model = Matcher(input_data=df_i_signupmonth, outcome=outcome, treatment=treatment,
                info_col=info_col, group_col=group_col)

In [11]:
features_importance = model.lama_feature_select()
features_importance

,Feature,Importance
0,pre_spends,259460.041138
3,industry,259460.041138
1,age,153843.079468
2,is_male,29594.749756


In [12]:
features = features_importance['Feature'].to_list()

In [13]:
# You can use both variant that you like. So you can just list the features to be matched

#results, quality_results = model.estimate(features=features_importance[:3])
results, quality_results, df_matched = model.estimate(features=features[:3])

  0%|          | 0/4 [00:00<?, ?it/s]

In [14]:
df_matched

,index,pre_spends,age,is_male,industry,pre_spends_matched,age_matched,is_male_matched,index_matched,industry_matched,post_spends,post_spends_matched,post_spends_matched_bias,treat,treat_matched
0,5030,499.0,46,0,1,499.5,47.0,1.0,[1926],1,530.000000,416.555556,113.424881,1,0
1,5031,487.5,60,0,1,487.0,60.0,0.5,"[746, 2361]",1,520.000000,422.277778,97.717321,1,0
2,5032,475.5,42,1,1,475.5,42.0,1.0,"[795, 1052, 2158]",1,517.888889,423.851852,94.037037,1,0
3,5033,472.5,64,1,1,471.5,64.0,1.0,[1637],1,516.666667,419.000000,97.656864,1,0
4,5034,499.5,36,0,1,499.0,35.0,0.5,"[973, 2256]",1,525.333333,417.000000,108.352897,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5025,5025,496.0,53,1,2,494.5,51.0,0.0,[303],2,406.222222,524.777778,118.448804,0,1
5026,5026,502.5,21,0,2,500.0,21.0,0.0,[302],2,423.333333,523.888889,100.411990,0,1
5027,5027,475.5,58,1,2,474.0,59.0,1.0,[405],2,431.444444,519.666667,88.146389,0,1
5028,5028,491.0,23,1,2,491.5,23.0,1.0,[345],2,408.555556,508.666667,100.139824,0,1


## 3. Results  
### 3.1 ATE, ATT, ATC

In [15]:
# model.matcher.results
results

,effect_size,std_err,p-val,ci_lower,ci_upper
ATE,99.817682,0.558638,0.0,98.722751,100.912613
ATC,99.838720,0.568383,0.0,98.724690,100.952751
ATT,99.579877,0.681198,0.0,98.244728,100.915025


### 3.2 SMD, PSI, KS-test, repeats

In [16]:
# matching quality result - SMD
model.quality_result

{'psi':         column_treated  anomaly_score_treated check_result_treated  \
 0          age_treated                   0.00                   OK   
 1     industry_treated                   0.00                   OK   
 2  post_spends_treated                  16.11                  NOK   
 3   pre_spends_treated                   0.01                   OK   
 
         column_untreated  anomaly_score_untreated check_result_untreated  
 0          age_untreated                     0.01                     OK  
 1     industry_untreated                     0.00                     OK  
 2  post_spends_untreated                     8.22                    NOK  
 3   pre_spends_untreated                     0.01                     OK  ,
 'ks_test':             match_control_to_treat  match_treat_to_control
 age                        1.00000                0.295413
 is_male                    0.14888                0.047913
 pre_spends                 1.00000                0.580909,
 's

In [17]:
# matching quality result - PSI
model.quality_result['psi']

,column_treated,anomaly_score_treated,check_result_treated,column_untreated,anomaly_score_untreated,check_result_untreated
0,age_treated,0.00,OK,age_untreated,0.01,OK
1,industry_treated,0.00,OK,industry_untreated,0.00,OK
2,post_spends_treated,16.11,NOK,post_spends_untreated,8.22,NOK
3,pre_spends_treated,0.01,OK,pre_spends_untreated,0.01,OK


In [18]:
# matching quality result - KS-test

model.quality_result['ks_test']

,match_control_to_treat,match_treat_to_control
age,1.00000,0.295413
is_male,0.14888,0.047913
pre_spends,1.00000,0.580909


In [19]:
# matching quality result - repeats
model.quality_result['repeats']

{'match_control_to_treat': 0.95, 'match_treat_to_control': 0.09}

### 3.3 Validation
validate result with placebo treatment  
our new effect size is close to zero (it should be)

In [20]:
 
model.validate_result()
 

  0%|          | 0/10 [00:00<?, ?it/s]

{'post_spends': [99.70039304957497, 0.1512325963149408]}